In [70]:
from sklearn.linear_model import LinearRegression
import numpy as np

X = np.array([[5.0, 3.4, 1.5, 0.2], [6.1, 2.8, 4.7, 1.2], [6.9, 3.1, 5.1, 2.3], [5.8, 2.7, 4.1, 1.0]])
y = np.array([[2], [1], [3], [1]])

In [71]:
lin_reg = LinearRegression(fit_intercept = False)
lin_reg.fit(X, y)
print(lin_reg.coef_)

print(lin_reg.intercept_)

[[ 0.04312268  0.75390335 -0.76728625  1.86022305]]
0.0


In [72]:
import numpy as np
import pandas as pd

housing = pd.read_csv('https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv')
housing.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


In [73]:
median = housing["total_bedrooms"].median()
housing["total_bedrooms"] = housing["total_bedrooms"].fillna(median)

In [74]:
housing["rooms_per_household"] = housing["total_rooms"] / housing["households"]

In [75]:
housing['bedromms_per_room'] = housing['total_bedrooms'] / housing['total_rooms']
del housing["total_bedrooms"], housing['total_rooms']

In [76]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

col_list = list(housing)
col_list.remove("ocean_proximity")
col_list.remove("median_house_value")

housing_numeric = housing[col_list]
housing_scaled = scaler.fit_transform(housing_numeric)
housing_scaled_df = pd.DataFrame(housing_scaled, index = housing_numeric.index, columns = housing_numeric.columns)

housing = pd.concat([housing_scaled_df, housing["median_house_value"], housing["ocean_proximity"]], axis = 1)

In [77]:
housing = pd.get_dummies(housing)
housing.head(10)

,longitude,latitude,housing_median_age,population,households,median_income,rooms_per_household,bedromms_per_room,median_house_value,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-1.327835,1.052548,0.982143,-0.974429,-0.977033,2.344766,0.628559,-1.029988,452600.0,0,0,0,1,0
1,-1.322844,1.043185,-0.607019,0.861439,1.669961,2.332238,0.327041,-0.888897,358500.0,0,0,0,1,0
2,-1.332827,1.038503,1.856182,-0.820777,-0.843637,1.782699,1.155620,-1.291686,352100.0,0,0,0,1,0
3,-1.337818,1.038503,1.856182,-0.766028,-0.733781,0.932968,0.156966,-0.449613,341300.0,0,0,0,1,0
4,-1.337818,1.038503,1.856182,-0.759847,-0.629157,-0.012881,0.344711,-0.639087,342200.0,0,0,0,1,0
5,-1.337818,1.038503,1.856182,-0.894071,-0.801787,0.087447,-0.269730,0.275563,269700.0,0,0,0,1,0
6,-1.337818,1.033821,1.856182,-0.292712,0.037823,-0.111366,-0.200918,-0.320242,299200.0,0,0,0,1,0
7,-1.337818,1.033821,1.856182,-0.237079,0.385698,-0.395137,-0.255232,0.115458,241400.0,0,0,0,1,0
8,-1.342809,1.033821,1.061601,-0.193810,0.249687,-0.942359,-0.458703,0.712372,226700.0,0,0,0,1,0
9,-1.337818,1.033821,1.856182,0.110844,0.560944,-0.094470,-0.185283,-0.223507,261100.0,0,0,0,1,0


In [78]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size = 0.2, random_state = 45)

print('# of train_set : %.0f, # of test_set : %.0f' %(train_set.shape[0], test_set.shape[0]))

# of train_set : 16512, # of test_set : 4128


In [79]:
train_set_features = train_set.drop('median_house_value', axis = 1)
train_set_target = train_set["median_house_value"].copy()

In [80]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

lin_reg = LinearRegression()
lin_reg.fit(train_set_features, train_set_target)

print("Learned Parameters : ")
print("Coefficients : ", lin_reg.coef_)
print("blas : ", lin_reg.intercept_)

Learned Parameters : 
Coefficients :  [-55774.14086907 -56964.69903623  12971.17103303 -53562.80929239
  57272.02939275  74147.51210095   6564.46012412  10459.02523705
 -25921.29448802 -63911.36887214 143057.05886283 -30846.20870803
 -22378.18679464]
blas :  244824.9222886056


In [81]:
test_set_features = test_set.drop("median_house_value", axis =1)
test_set_target = test_set["median_house_value"].copy()

final_model = lin_reg
final_predictions = final_model.predict(test_set_features)

print("Test_r2_score : ", r2_score(test_set_target, final_predictions))

delta = test_set_target - final_predictions
print("Errors (%) in housing value prediction : ", np.mean(np.abs(delta)/test_set_target))

final_mse = mean_squared_error(test_set_target, final_predictions)
final_rmse = np.sqrt(final_mse)

Test_r2_score :  0.6308608545588804
Errors (%) in housing value prediction :  0.2861199789930685


In [82]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [83]:
model_ridge = Ridge(alpha = 100)
model_ridge.fit(train_set_features, train_set_target)

Ridge(alpha=100, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [84]:
print("Train_score : ", model_ridge.score(train_set_features, train_set_target))

Train_score :  0.652511581004448


In [85]:
ridge_predicted = model_ridge.predict(test_set_features)
print("Test_score : ", r2_score(test_set_target, ridge_predicted))

Test_score :  0.6333956237676854


In [86]:
delta = test_set_target - ridge_predicted
print('Errors in housing value prediction : ', np.mean(np.abs(delta)/test_set_target))

Errors in housing value prediction :  0.28415587884460786


In [87]:
final_ridge_mse = mean_squared_error(test_set_target, ridge_predicted)
final_ridge_rmse = np.sqrt(final_ridge_mse)
print("final_ridge_RMSE : ", final_ridge_rmse)

final_ridge_RMSE :  69746.54943237505


In [88]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [89]:
from sklearn.linear_model import Lasso

In [90]:
model_lasso = Lasso(alpha= 1000)
model_lasso.fit(train_set_features, train_set_target)
print("Train_score : ", model_lasso.score(train_set_features, train_set_target))

Train_score :  0.6477400839070926


In [91]:
lasso_predicted = model_lasso.predict(test_set_features)
print("Test_score : ", r2_score(test_set_target, lasso_predicted))

Test_score :  0.6353348735073737


In [92]:
delta = test_set_target - lasso_predicted
print("Errors in housing value prediction : ",np.mean(np.abs(delta)/test_set_target))

Errors in housing value prediction :  0.28205827189150406


In [93]:
final_lasso_mse = mean_squared_error(test_set_target, lasso_predicted)
final_lasso_rmse = np.sqrt(final_lasso_mse)
print("RMSE is : ", final_lasso_rmse)

RMSE is :  69561.83352439076


In [94]:
import numpy as np
import pandas as pd

In [95]:
housing = pd.read_csv('housing.csv')

In [96]:
housing.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


In [97]:
median = housing["total_bedrooms"].median()
housing["total_bedrooms"] = housing["total_bedrooms"].fillna(median)

In [98]:
housing['bedrooms_per_romms'] = housing['total_bedrooms'] / housing['total_rooms']
del housing["total_bedrooms"], housing['total_rooms']

In [99]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

col_list = list(housing)
col_list.remove("ocean_proximity")
col_list.remove("median_house_value")

housing_numeric = housing[col_list]
housing_scaled = scaler.fit_transform(housing_numeric)

housing_scaled_df = pd.DataFrame(housing_scaled, index = housing_numeric.index, columns=housing_numeric.columns)

housing = pd.concat([housing_scaled_df, housing["median_house_value"], housing["ocean_proximity"]], axis=1)
housing.head()

,longitude,latitude,housing_median_age,population,households,median_income,bedrooms_per_romms,median_house_value,ocean_proximity
0,-1.327835,1.052548,0.982143,-0.974429,-0.977033,2.344766,-1.029988,452600.0,NEAR BAY
1,-1.322844,1.043185,-0.607019,0.861439,1.669961,2.332238,-0.888897,358500.0,NEAR BAY
2,-1.332827,1.038503,1.856182,-0.820777,-0.843637,1.782699,-1.291686,352100.0,NEAR BAY
3,-1.337818,1.038503,1.856182,-0.766028,-0.733781,0.932968,-0.449613,341300.0,NEAR BAY
4,-1.337818,1.038503,1.856182,-0.759847,-0.629157,-0.012881,-0.639087,342200.0,NEAR BAY


In [100]:
 housing = pd.get_dummies(housing)
 housing.head(10)

,longitude,latitude,housing_median_age,population,households,median_income,bedrooms_per_romms,median_house_value,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-1.327835,1.052548,0.982143,-0.974429,-0.977033,2.344766,-1.029988,452600.0,0,0,0,1,0
1,-1.322844,1.043185,-0.607019,0.861439,1.669961,2.332238,-0.888897,358500.0,0,0,0,1,0
2,-1.332827,1.038503,1.856182,-0.820777,-0.843637,1.782699,-1.291686,352100.0,0,0,0,1,0
3,-1.337818,1.038503,1.856182,-0.766028,-0.733781,0.932968,-0.449613,341300.0,0,0,0,1,0
4,-1.337818,1.038503,1.856182,-0.759847,-0.629157,-0.012881,-0.639087,342200.0,0,0,0,1,0
5,-1.337818,1.038503,1.856182,-0.894071,-0.801787,0.087447,0.275563,269700.0,0,0,0,1,0
6,-1.337818,1.033821,1.856182,-0.292712,0.037823,-0.111366,-0.320242,299200.0,0,0,0,1,0
7,-1.337818,1.033821,1.856182,-0.237079,0.385698,-0.395137,0.115458,241400.0,0,0,0,1,0
8,-1.342809,1.033821,1.061601,-0.193810,0.249687,-0.942359,0.712372,226700.0,0,0,0,1,0
9,-1.337818,1.033821,1.856182,0.110844,0.560944,-0.094470,-0.223507,261100.0,0,0,0,1,0


In [101]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size = 0.2, random_state=45)

print('# of train_set : %.0f, # of test_set : %0.f'%(train_set.shape[0], test_set.shape[0]))

# of train_set : 16512, # of test_set : 4128


In [102]:
train_set_features = train_set.drop('median_house_value',axis = 1)
train_set_target = train_set["median_house_value"].copy()

In [103]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

lin_reg = LinearRegression()
lin_reg.fit(train_set_features, train_set_target)

print("Learned Parameters : ")
print("Coefficients: ", lin_reg.coef_)
print("bias: ", lin_reg.intercept_)

print("Train_r2_score : ", lin_reg.score(train_set_features, train_set_target))

Learned Parameters : 
Coefficients:  [-50759.39036387 -51574.01642357  12545.81188557 -53652.59414589
  56812.99611044  75937.26906977   9146.7447002  -26966.35524515
 -65564.17761279 146019.09338452 -31175.40269955 -22313.15782703]
bias:  245849.07939194865
Train_r2_score :  0.6506911525314935


In [105]:
test_set_features = test_set.drop("median_house_value",axis=1)
test_set_target = test_set["median_house_value"].copy()

final_model = lin_reg
final_predictions = final_model.predict(test_set_features)

print("Test_r2_score: ", r2_score(test_set_target, final_predictions))

delta = test_set_target - final_predictions

final_mse = mean_squared_error(test_set_target, final_predictions)
final_rmse = np.sqrt(final_mse)
print("RMSE is : ",final_rmse)

Test_r2_score:  0.6279901608184868
RMSE is :  70258.86302412274


In [107]:
print(test_set_features.iloc[2])
print(test_set_features.iloc[43])

x1 = np.array(test_set_features.iloc[2])
x2 = np.array(test_set_features.iloc[43])
prediction1 = np.dot(lin_reg.coef_,x1) + lin_reg.intercept_
prediction2 = np.dot(lin_reg.coef_,x2) + lin_reg.intercept_

print("\n2th sample- predicted: {}, Real value: {}".format(prediction1,test_set_target.iloc[2]))
print("43rd sample- predicted: {}, Real value: {}".format(prediction2,test_set_target.iloc[43]))

longitude                     0.813436
latitude                     -0.792107
housing_median_age            0.108104
population                   -0.512592
households                   -0.498377
median_income                 0.336318
bedrooms_per_romms           -0.328572
ocean_proximity_<1H OCEAN     1.000000
ocean_proximity_INLAND        0.000000
ocean_proximity_ISLAND        0.000000
ocean_proximity_NEAR BAY      0.000000
ocean_proximity_NEAR OCEAN    0.000000
Name: 10112, dtype: float64
longitude                    -0.728878
latitude                      1.642463
housing_median_age           -1.481058
population                   -0.132879
households                    0.029976
median_income                 0.108870
bedrooms_per_romms           -0.717413
ocean_proximity_<1H OCEAN     0.000000
ocean_proximity_INLAND        1.000000
ocean_proximity_ISLAND        0.000000
ocean_proximity_NEAR BAY      0.000000
ocean_proximity_NEAR OCEAN    0.000000
Name: 9995, dtype: float64

2th samp

In [109]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

model_ridge = Ridge(alpha = 100)
model_ridge.fit(train_set_features, train_set_target)

print("Train_score : ", model_ridge.score(train_set_features, train_set_target))

ridge_predicted = model_ridge.predict(test_set_features)

print("Test_score : ", r2_score(test_set_target, ridge_predicted))

delta = test_set_target - ridge_predicted
print("Errors in housing value preection : ", np.mean(np.abs(delta)/test_set_target))

final_ridge_mse = mean_squared_error(test_set_target, ridge_predicted)
final_ridge_rmse = np.sqrt(final_ridge_mse)
print("final_ridge_RMSE : ", final_ridge_rmse)

import warnings
warnings.filterwarnings(action = 'ignore')

Train_score :  0.6498504445520576
Test_score :  0.6305114538128149
Errors in housing value preection :  0.2844805647110943
final_ridge_RMSE :  70020.36886559935


In [110]:
from sklearn.linear_model import Lasso

model_lasso = Lasso(alpha = 1000)
model_lasso.fit(train_set_features, train_set_target)

print("Train_score : ", model_lasso.score(train_set_features, train_set_target))

lasso_predicted = model_lasso.predict(test_set_features)
print("Test_score : ", r2_score(test_set_target, lasso_predicted))

delta = test_set_target - lasso_predicted

final_lasso_mse = mean_squared_error(test_set_target, lasso_predicted)
final_lasso_rmse = np.sqrt(final_lasso_mse)
print("RMSE is : ", final_lasso_rmse)

Train_score :  0.6455243865804163
Test_score :  0.633200443639562
Errors in housing value prediction :  0.2827909972538183
RMSE is :  69765.11348931272
